In [1]:
'''
this one is designed for 3rd round 2021
'''
import csv
import docx
from tqdm import tqdm
import utils
import xlrd

In [2]:
# config
xlsx_path = '/mnt/c/Users/gray/seadrive_root/mail_1/我的资料库/mine/STANDARD/第三次会议/11.17版本 python自动化merge的base.xlsx'

# in_path = '/home/gray/desktop/STANDARD/test2.docx' # example
in_path = '/mnt/c/Users/gray/seadrive_root/mail_1/我的资料库/mine/STANDARD/第三次会议/11.18 2.docx'

out_path = '/mnt/c/Users/gray/seadrive_root/mail_1/我的资料库/mine/STANDARD/第三次会议/out.docx'

doc = docx.Document(in_path)

In [3]:
replace = {}
workbook = xlrd.open_workbook(filename=xlsx_path)
worksheet = workbook.sheet_by_name(workbook.sheet_names()[0])

In [4]:
for i in tqdm(range(3, worksheet.nrows)):
    
    row = worksheet.row_values(i)
    
    # add auto-summit flag in row[0] so having increment all index
    sec_num = utils.get_sec_num_new(str(row[2]),from_head=True)
    pre_result = row[8]  # 0 -> deny, 1-> yes
    discuss = row[9]  # discuss flag
    
    if pre_result == '' and discuss == '':
        replace[row[3]] = row[4]

100%|█████████████████████████████████████████████████████████████████████████████| 603/603 [00:00<00:00, 402093.05it/s]


In [5]:
list(replace.items())[:2]

[('插入到底板或是母板的系统应该使用主机硬件RAID。',
  '插入到背板或是主板的系统应该使用“存储管理技术规范，第7部分 主机元素1.8.0版本4中的8 主机硬件RAID控制器配置文件。'),
 ('目标端口通过那些对主机有用的逻辑单元数(LUN)来组成配置文件模型的端口部分(例如，iSCSI的光纤通道)。',
  '目标端口组件配置文件对端口（例如，是光纤通道还是iSCSI）进行建模，逻辑单元号(LUN)通过端口为主机所使用。')]

In [6]:
# replace没有区分正文和表格，做个safety check
track_replace = {}
for k in replace.keys():
    track_replace[k] = 0

In [7]:
# replace in text paragraphs
for para in tqdm(list(doc.paragraphs)):
    for a,b in replace.items():
        if a != '' and a in para.text:
            utils.replace_para_text(para,a,b)
            track_replace[a] += 1

100%|████████████████████████████████████████████████████████████████████████████| 12054/12054 [00:20<00:00, 594.75it/s]


In [8]:
# replace in tables
for table_index in tqdm(range(len(list(doc.tables)))):
    table = doc.tables[table_index]
    for row in table.rows:
        for cell in row.cells:
            for para in cell.paragraphs:
                for a,b in replace.items():
                    if a!='' and a in para.text: # do replace
                        utils.replace_para_text(para,a,b)
                        track_replace[a] += 1

100%|█████████████████████████████████████████████████████████████████████████████████| 723/723 [00:41<00:00, 17.52it/s]


In [9]:
# check 0
for k,v in track_replace.items():
    if v != 1:
        print(k,v)
        print('--------------------')

4.6.5 CIM_ProtocolControllerForUnit项(从所有逻辑单元数来看任意的逻辑单元) 0
--------------------
 0
--------------------
应使用元素一致性文件关联与注册的配置文件关联 0
--------------------
每个存储容量存储设备或应用程序块都有一个状态。存储空间和逻辑盘中所描述的存储内容，是与块不同的集团 0
--------------------
存储空间是存储容量，须从制度暴露通过外部接口分配 0
--------------------
9.1.23瘦供给元素 0
--------------------
这个版本用存储复制性能性能类实例来说明复制服务性能 0
--------------------
客户应参考25.3，了解利用存储分层服务的步骤列表。 0
--------------------
错误!未找到引用源。显示此属性的可能值。 0
--------------------
错误!未找到引用源。总结了外部方法： 0
--------------------


In [10]:
len(track_replace)

174

In [11]:
def is_zero(n):
    return n == 0

tmp_zero = list(filter(is_zero,[v for v in track_replace.values()]))
print(len(tmp_zero))

10


In [12]:
doc.save(out_path)